In [1]:
# general package
from datetime import datetime
import logging
import sys
# Transform
import pandas as pd
from pandas import DataFrame
# mysql.connector
import pymysql
# airflow package
# from airflow import DAG
# from airflow.operators.python import PythonOperator

In [242]:
# !conda install -c conda-forge -y --prefix {sys.prefix} pandas
# !conda install -c conda-forge -y --prefix {sys.prefix} sqlalchemy
# !conda install -c conda-forge -y --prefix {sys.prefix} pymysql
# !conda install -c conda-forge -y --prefix {sys.prefix} ipython-sql

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [2]:
def _get_MySQL_Connection():
    import os
    import configparser
    from pathlib import Path
    sys.path.append(os.getcwd())
    
    config = configparser.ConfigParser()
    config.read(Path("./config/config.ini"),encoding='utf-8')

    user = config['aws_ec2_mysql']['user']
    password = config['aws_ec2_mysql']['password']
    host = config['aws_ec2_mysql']['host']
    port = config['aws_ec2_mysql']['port']
    dbname = config['aws_ec2_mysql']['dbname']
    charset = config['aws_ec2_mysql']['charset']

    try:
        conn = pymysql.connect(
            host=host,
            port=int(port),
            user=user,
            password=password,
            db=dbname,
            charset=charset
        )
        logging.info('DB Connect : ',datetime.now().strftime('%Y.%m.%d.%H.%M') )
        return conn.cursor()
    except Exception as e:
        logging.error('DB Connect Error : ',datetime.now().strftime('%Y.%m.%d.%H.%M') )
        logging.error(e.message)
        sys.exit(5)
#        session.rollback()
    #connection_string

In [3]:
def DataToCSV(df):
    import os
    
    path = os.getcwd()
    mkdirPath = path + "/data/mysqlData"
    if not os.path.exists(mkdirPath):
        os.mkdir(mkdirPath)
    
    now_date = datetime.now().strftime('%Y.%m.%d.%H.%M')
    file_name = now_date+"_output.csv"
    
    output_file = mkdirPath+"/"+file_name
    
    df.to_csv(output_file,index=False)
    
    if os.path.isfile(output_file) :
        logging.info('File Created: ', output_file)

In [4]:
def load():
    cur = _get_MySQL_Connection()
    sql = "select name, email from member;"
    cur.execute(sql)
    result = cur.fetchall()
    df = DataFrame(result)    
    
    return df

In [5]:
def etl():
    df = load()
    DataToCSV(df)

In [6]:
etl()

--- Logging error ---
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/pipeline/lib/python3.9/site-packages/pymysql/connections.py", line 613, in connect
    sock = socket.create_connection(
  File "/opt/homebrew/anaconda3/envs/pipeline/lib/python3.9/socket.py", line 844, in create_connection
    raise err
  File "/opt/homebrew/anaconda3/envs/pipeline/lib/python3.9/socket.py", line 832, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/folders/0j/gzlqhwtn7gg81y1sxjmmbv8m0000gn/T/ipykernel_11273/483125701.py", line 18, in _get_MySQL_Connection
    conn = pymysql.connect(
  File "/opt/homebrew/anaconda3/envs/pipeline/lib/python3.9/site-packages/pymysql/connections.py", line 353, in __init__
    self.connect()
  File "/opt/homebrew/anaconda3/envs/pipeline/lib/python3.9/site-packages/pymysql/connections.py", line 664, in connect
    

AttributeError: 'OperationalError' object has no attribute 'message'

In [239]:
dag_second_assingnment = DAG(
    dag_id='MySQLToCsvFile',
    catchup = False,
    start_date = datetime(2022,1,10),
    schedule_interval = '0 20 * * *'
)

NameError: name 'DAG' is not defined

In [ ]:
task = PythonOperator(
    task_id = 'MySQLTOCsv',
    python_callable = etl,
    dag = dag_second_assingnment
)